# An Introduction to Using PyBEN

This is a small tutorial file that is meant to help users get to using PyBEN: the python interface
for the [binary-ensemble](https://crates.io/binary-ensemble) Rust package.

### Things go here

My goal for right now is to see if the interface is intuitive and to see how well my 
development scripts are working and how much the documentation strings tell you. So.. 